# Variational Quantum Linear Solver for Newton-Raphson 
As explained in the previous notebook, the Newton Raphson approach relies on the solution of a linear system. Several quantum linear solvers have been developped, one of them being the Variational Quantum linear solver. This solver uses a variational approach to solve the system of linear equations.

## Example 
To make things clearer let's consider the following example:

$$
f_1(x,y) = 0.2 x^3 - 0.4 y \\
f_2(x,y) = - 0.4 x  + 0.8 y^2 \\
$$

That we define in the following function

In [ ]:
import numpy as np
def func(input):
    def f1(x,y):
        return 0.1*(2*x**3 - 4* y)
    def f2(x,y):
        return 0.1*(- 4 * x + 8*y**2)
    x, y = input
    return np.array([f(x,y) for f in[f1,f2]])

# Using VQSL to solve the linear system

The `QuantumNewtonRaphson` can use our `vqls` solver to solve the linear system of equations at the hart of the newton raphson routine.  

In [ ]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import CG
from quantum_newton_raphson.newton_raphson import newton_raphson
from quantum_newton_raphson.vqls_solver import VQLS_SOLVER

estimator = Estimator()
ansatz = RealAmplitudes(1, entanglement='full', reps=3)
optim = CG(maxiter=1000, tol=1E-10)

# define the linear solver with the reorder solver
solver = VQLS_SOLVER(estimator=estimator, ansatz=ansatz, optimizer=optim, matrix_decomposition="symmetric")

# Solve the newton raphson problem
res = newton_raphson(func, np.random.rand(2), linear_solver=solver, verbose=True, max_iter=10)


We can check the solution of the linear system within he NR step obtained with VQLS for one of the iteration

In [ ]:
import matplotlib.pyplot as plt
n = 4
plt.subplot(121)
plt.semilogy(res.linear_solver_results[n].logger.values)
plt.subplot(122)
plt.scatter(res.linear_solver_results[n].ref, res.linear_solver_results[n].solution)
_ = plt.axline( (0,0), slope=1,linestyle='--',color='grey')


We can check the solution against the solution obatined via a classical newton raphson 

In [ ]:
import matplotlib.pyplot as plt
res_ref = newton_raphson(func, np.random.rand(2))
plt.scatter(res_ref.solution, res.solution)
_ = plt.axline( (0,0),slope=1,linestyle='--',color='grey')